# SRL GRPO Training (Colab, A100)
End-to-end notebook to build SRL data, split 95/5, and train with GRPO + LoRA on an A100.

**Memory-optimized configuration for single GPU training.**

In [1]:
# ============================================================================
# Setup and Installation (vLLM Edition - 6x faster GRPO training)
# ============================================================================
# Runtime: GPU (A100 recommended)
# Make sure to select GPU: Runtime -> Change runtime type -> GPU

import os
from pathlib import Path

# Repository configuration
REPO_URL = "https://github.com/iroblesrazzaq/SRL-reasoning.git"
BRANCH = "main"
WORKDIR = "/content/SRL-reasoning"

# Install vLLM + latest TRL (with vLLM support for fast generation)
# vLLM provides 6-10x speedup for GRPO by optimizing the generation bottleneck
!pip install -q git+https://github.com/huggingface/trl.git  # Latest TRL with vLLM support
!pip install -q vllm bitsandbytes datasets peft accelerate

# Clone repo if not exists
if not os.path.exists(WORKDIR):
    !git clone --branch $BRANCH $REPO_URL $WORKDIR

%cd $WORKDIR
!git pull

# Install package
!pip install -e . --no-deps

# Verify GPU
import torch
print("=" * 80)
print("SETUP COMPLETE (vLLM Edition)")
print("=" * 80)
print(f"✓ Repository: {WORKDIR}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✓ GPU Memory: {gpu_mem:.2f} GB")
print("=" * 80)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 149.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 20.2 MB/s eta 0:00:00
Cloning into '/content/SRL-reasoning'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 401 (delta 65), reused 72 (delta

In [2]:
# Verify vLLM and TRL installation
from inspect import signature
from trl import GRPOConfig

# Check vLLM support in TRL
supported = set(signature(GRPOConfig.__init__).parameters)
assert 'use_vllm' in supported, "TRL version doesn't support vLLM, update TRL"
print("✓ vLLM support verified in TRL")

# Print versions
import torch
import trl
import vllm
print(f"TRL version: {trl.__version__}")
print(f"vLLM version: {vllm.__version__}")
print(f"PyTorch {torch.__version__}, CUDA {torch.version.cuda}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth version: 2025.11.6
PyTorch 2.9.0+cu126, CUDA 12.6


In [3]:
#@title 1. Global config
import random, numpy as np
from pathlib import Path
import gc

SEED = 42
BASE_MODEL = 'Qwen/Qwen3-4B-Instruct-2507'
REPO_DIR = Path('/content/SRL-reasoning')
OUTPUT_DIR = Path('/content/outputs/srl_grpo')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR = REPO_DIR / 'data'
DATA_DIR.mkdir(parents=True, exist_ok=True)

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
print('DATA_DIR:', DATA_DIR)

DATA_DIR: /content/SRL-reasoning/data


In [4]:
#@title 2. Build SRL data (s1K-1.1 -> step-wise JSONL)
from src.shared.build_srl_data import load_teacher_dataset, normalize_dataset, build_srl_dataset, save_jsonl
from src.shared.splits import split_by_trajectory

raw_ds = load_teacher_dataset('simplescaling/s1K-1.1', split='train')
norm_trajs = normalize_dataset(raw_ds)
srl_examples = build_srl_dataset(norm_trajs)

all_path = DATA_DIR / 'srl_steps.jsonl'
save_jsonl(srl_examples, all_path)

train_examples, val_examples, _ = split_by_trajectory(
    str(all_path),
    train_ratio=0.95,
    val_ratio=0.05,
    test_ratio=0.0,
    seed=SEED,
)

train_path = DATA_DIR / 'train.jsonl'
val_path = DATA_DIR / 'val.jsonl'
save_jsonl(train_examples, train_path)
save_jsonl(val_examples, val_path)

print(f'Train examples: {len(train_examples)}')
print(f'Val examples:   {len(val_examples)}')

Loading dataset: simplescaling/s1K-1.1 (split: train)...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/22.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Loaded 1000 examples


Saving to JSONL: 100%|██████████| 143/143 [00:00<00:00, 42366.71example/s]

Train examples: 2528
Val examples:   143


In [5]:
#@title 3. Prepare HF datasets for GRPO
from scripts.train_srl import load_srl_dataset

train_dataset = load_srl_dataset(str(train_path))
val_dataset = load_srl_dataset(str(val_path))

print(f'HF datasets -> train {len(train_dataset)}, val {len(val_dataset)}')

✓ Unsloth available - using optimized training
HF datasets -> train 2528, val 143


In [6]:
#@title 4. Load model with transformers + PEFT (vLLM handles generation)
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

# Clear any existing GPU memory
gc.collect()
torch.cuda.empty_cache()

# Load base model with standard transformers
# vLLM will handle fast generation, so we use standard loading here
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)

# Set padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

# Apply LoRA with standard PEFT
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    task_type=TaskType.CAUSAL_LM,
    bias="none",
)
model = get_peft_model(model, lora_config)
model.enable_input_require_grads()
model.gradient_checkpointing_enable()

# Print memory usage
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f'\n✓ Model loaded with PEFT LoRA!')
print(f'  Trainable params: {trainable_params/1e6:.1f}M / {total_params/1e6:.1f}M')
if torch.cuda.is_available():
    print(f'  GPU Memory used: {torch.cuda.memory_allocated(0)/1e9:.2f} GB')

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.6 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.



✓ Unsloth model loaded!
  Trainable params: 33.0M / 4055.5M
  GPU Memory used: 8.20 GB


In [7]:
#@title 5. Configure GRPO trainer (vLLM-optimized settings)
from inspect import signature
from trl import GRPOConfig
from scripts.train_srl import SRLGRPOTrainer, create_reward_function

reward_fn = create_reward_function(tokenizer)

# Check which parameter names are supported
supported = set(signature(GRPOConfig.__init__).parameters)

# ============================================================================
# VLLM-OPTIMIZED GRPO CONFIGURATION
# ============================================================================
# vLLM provides 6-10x speedup by optimizing the generation bottleneck
# (which is 90%+ of GRPO training time)
# ============================================================================

grpo_kwargs = {
    'output_dir': str(OUTPUT_DIR),

    # === ENABLE VLLM (CRITICAL FOR SPEEDUP) ===
    'use_vllm': True,
    'vllm_gpu_memory_utilization': 0.5,  # Leave 50% for training model

    # === TRAINING SCHEDULE (Paper settings) ===
    'num_train_epochs': 3,               # Start with 3 epochs for testing
    'learning_rate': 5e-7,               # Paper: 5e-7
    'warmup_ratio': 0.0,                 # Paper: no warmup
    'max_grad_norm': 1.0,                # Paper: 1.0

    # === BATCH SIZE (optimized for vLLM) ===
    'per_device_train_batch_size': 4,    # Increased from 1
    'num_generations': 4,                 # Reduced from 8 (still effective)
    'per_device_eval_batch_size': 4,     # Must be divisible by num_generations
    'gradient_accumulation_steps': 32,    # Effective batch ~128

    # === GRPO-SPECIFIC ===
    'beta': 0.0,                         # Paper: no KL penalty for SRL
    'temperature': 1.0,                  # Paper: 1.0 for rollouts

    # === TOKEN LIMITS (reduced for faster generation) ===
    'max_prompt_length': 512,            # Reduced from 1024
    'max_completion_length': 256,        # Reduced from 1024 (paper uses 256-512)

    # === CHECKPOINTING ===
    'save_strategy': 'epoch',
    'save_total_limit': 2,
    'load_best_model_at_end': True,
    'metric_for_best_model': 'eval_reward',
    'greater_is_better': True,

    # === OPTIMIZATION ===
    'optim': 'adamw_8bit',               # Memory-efficient optimizer
    'bf16': True,                        # bfloat16 precision

    # === LOGGING ===
    'logging_steps': 1,
    'report_to': 'none',
    'seed': SEED,
}

# Handle eval_strategy naming
if 'eval_strategy' in supported:
    grpo_kwargs['eval_strategy'] = 'epoch'
else:
    grpo_kwargs['evaluation_strategy'] = 'epoch'

# Filter to only supported parameters
grpo_config = GRPOConfig(**{k: v for k, v in grpo_kwargs.items() if k in supported})

print("=" * 80)
print("GRPO Config Summary (vLLM-Optimized)")
print("=" * 80)
print(f"  use_vllm: {grpo_kwargs['use_vllm']}")
print(f"  vllm_gpu_memory_utilization: {grpo_kwargs['vllm_gpu_memory_utilization']}")
print(f"  per_device_train_batch_size: {grpo_kwargs['per_device_train_batch_size']}")
print(f"  num_generations: {grpo_kwargs['num_generations']}")
print(f"  Sequences per step: {grpo_kwargs['per_device_train_batch_size']} x {grpo_kwargs['num_generations']} = {grpo_kwargs['per_device_train_batch_size'] * grpo_kwargs['num_generations']}")
print(f"  Effective batch size: {grpo_kwargs['per_device_train_batch_size']} x {grpo_kwargs['gradient_accumulation_steps']} = {grpo_kwargs['per_device_train_batch_size'] * grpo_kwargs['gradient_accumulation_steps']}")
print(f"  Max tokens: prompt={grpo_kwargs['max_prompt_length']}, completion={grpo_kwargs['max_completion_length']}")
print(f"  Total max sequence: {grpo_kwargs['max_prompt_length'] + grpo_kwargs['max_completion_length']}")
print("=" * 80)

GRPO Config Summary (Unsloth-Optimized)
  per_device_train_batch_size: 1
  num_generations: 8
  Sequences per step: 1 × 8 = 8
  Effective batch size: 1 × 512 = 512
  Max tokens: prompt=1024, completion=1024
  Total max sequence: 2048


In [8]:
#@title 6. Initialize trainer and start training

# Clear cache before training
gc.collect()
torch.cuda.empty_cache()

# Initialize trainer (vLLM handles generation automatically)
trainer = SRLGRPOTrainer(
    model=model,
    args=grpo_config,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    reward_funcs=reward_fn,
    filter_epsilon=1e-4,
)

print("\n" + "=" * 80)
print("STARTING TRAINING (vLLM-Optimized)")
print("=" * 80)
print(f"GPU Memory before training: {torch.cuda.memory_allocated(0)/1e9:.2f} GB")
print("vLLM will handle fast generation (6-10x speedup)")

train_result = trainer.train()
print(train_result)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

chat_template.jinja: 0.00B [00:00, ?B/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.



STARTING TRAINING (Unsloth-Optimized)
GPU Memory before training: 8.20 GB


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,528 | Num Epochs = 30 | Total steps = 1,170
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 512
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 512 x 1) = 512
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)
`generation_config` default values have been modified to match model-specific defaults: {'max_length': 262144, 'temperature': 0.7, 'top_p': 0.8}. If this is not desired, please set these values explicitly.


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.73 GiB. GPU 0 has a total capacity of 79.32 GiB of which 4.63 GiB is free. Process 3266 has 74.67 GiB memory in use. Of the allocated memory 60.10 GiB is allocated by PyTorch, and 14.06 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#@title 7. Save best model to Google Drive
from google.colab import drive

drive.mount('/content/drive')

BEST_DIR = Path('/content/drive/MyDrive/SRL_Best_Model')
BEST_DIR.mkdir(parents=True, exist_ok=True)

# Save using Unsloth's optimized save
model.save_pretrained(str(BEST_DIR))
tokenizer.save_pretrained(str(BEST_DIR))

print('Saved best model to', BEST_DIR)

In [ ]:
#@title 8. (Optional) Merge LoRA and save full model
# Uncomment to merge LoRA weights into the base model for easier deployment

# MERGED_DIR = Path('/content/drive/MyDrive/SRL_Merged_Model')
# MERGED_DIR.mkdir(parents=True, exist_ok=True)

# model.save_pretrained_merged(
#     str(MERGED_DIR),
#     tokenizer,
#     save_method="merged_16bit",  # Full 16-bit merged model
# )
# print('Saved merged model to', MERGED_DIR)